In [1]:
import os
import pybedtools
pybedtools.set_tempdir('./tmp/')
import pandas as pd
import time
import sys
import yaml

In [2]:
sites_yaml = '10000_unfiltered_sites_CIS_BP_v2.yaml'
num_sites = 10000
genome='hg38'
out_dir = '../overlaps_10000_sites/'

In [3]:
sites = yaml.safe_load(open(sites_yaml))
sites = pd.Series(sites['site_lists'])

In [4]:
print(len(sites))

377


In [5]:
os.getcwd()

'/fh/fast/ha_g/user/adoebley/projects/griffin_revisions_1/sites/TFBS/scripts'

In [6]:
start_time=time.time()
int_time=time.time()
for j in range(len(sites)):
    file_a = sites.iloc[j]
    name_a = sites.index[j]
    
    output=pd.DataFrame(columns=['TF_a','TF_b','num_site_TF_b','exact','window'])
    out_path=out_dir+'/'+name_a+'_top'+str(num_sites)+'_vs_all.txt'
    
    if os.path.isfile(out_path):
        print(j,'already done',name_a)
        continue
    print(name_a,file_a, time.time()-start_time)
    sys.stdout.flush()
    int_time=time.time()

    a = pd.read_csv(file_a, sep='\t')[['Chrom','position']] #necessary because I used an abnormal bed file header
    a = pybedtools.BedTool.from_dataframe(a[['Chrom','position','position']])
    extended_a=a.slop(genome=genome,b=250)


    for k in range(len(sites)):
        file_b = sites.iloc[k]
        name_b = sites.index[k]
        b = pd.read_csv(file_b, sep='\t')[['Chrom','position']] #necessary because I used an abnormal bed file header

        b = pybedtools.BedTool.from_dataframe(b[['Chrom','position','position']])
        extended_b=b.slop(genome=genome,b=250)
        new_line={'TF_a':name_a,'TF_b':name_b,'num_site_TF_b':len(b),'exact':(a+b).count(),'window':(extended_a+extended_b).count()}
        output=output.append(new_line,ignore_index=True)
        if k%50==0:
            print(k,new_line, time.time()-int_time)
    pybedtools.cleanup(remove_all=True)
    output= output.sort_values(by= ['exact','window'],ascending=False)
    output.to_csv(out_path,sep='\t', index=False)
    print('done',file_a, time.time()-start_time, time.time()-int_time)

0 already done AHR.hg38.10000
1 already done AR.hg38.10000
2 already done ARID3A.hg38.10000
3 already done ARNT.hg38.10000
4 already done ARNTL.hg38.10000
5 already done ASCL1.hg38.10000
6 already done ASCL2.hg38.10000
7 already done ATF1.hg38.10000
8 already done ATF2.hg38.10000
9 already done ATF3.hg38.10000
10 already done ATF4.hg38.10000
11 already done ATF7.hg38.10000
12 already done BACH1.hg38.10000
13 already done BACH2.hg38.10000
14 already done BATF.hg38.10000
15 already done BATF3.hg38.10000
16 already done BCL11A.hg38.10000
17 already done BCL11B.hg38.10000
18 already done BCL6.hg38.10000
19 already done BHLHE40.hg38.10000
20 already done BPTF.hg38.10000
21 already done CDX2.hg38.10000
22 already done CEBPA.hg38.10000
23 already done CEBPB.hg38.10000
24 already done CEBPD.hg38.10000
25 already done CEBPG.hg38.10000
26 already done CLOCK.hg38.10000
27 already done CREB1.hg38.10000
28 already done CREM.hg38.10000
29 already done CTCF.hg38.10000
30 already done CTCFL.hg38.10000

In [7]:
pybedtools.cleanup(remove_all=True)